### 1. Diet dataset - 분산 분석의  full process 적용 및 결과 해석  (다이어트 효과 분석) 

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../data/diet.csv")
data

,gender,Diet,weight,weight6weeks
0,M,B,60,60.0
1,M,B,103,103.0
2,F,A,58,54.2
3,F,A,60,54.0
4,F,A,64,63.3
...,...,...,...,...
73,M,C,83,80.2
74,M,C,84,79.9
75,M,C,85,79.7
76,M,C,87,77.8
